In [440]:
#standard packages
import pandas as pd
import numpy as np

#webscraping packages & SQLite
import pymongo
from pymongo import MongoClient
from bs4 import BeautifulSoup, SoupStrainer
import requests
import urllib.request
import functions as mf

#nlp packages
import string
import nltk
from nltk.corpus import stopwords
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors 
from gensim.scripts.glove2word2vec import glove2word2vec
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_lg')

#EDA packages
from collections import Counter, OrderedDict
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

#flask
from flask import Flask, render_template, url_for 
from forms import RegistrationForm

In [2]:
client = MongoClient()
db = client.case_files
cases = db.cases

In [913]:
df = pd.DataFrame(list(cases.find()))

In [914]:
c_d = mf.clean_data

In [915]:
clean_data_df = c_d.clean_data(df)

In [916]:
clean_data_df.head()

,_id,attorneys,case_id,case_name,case_text,court_id,court_name,decision_date,frontend_url,judges,majority_opinion,other_opinions,tokenized_majority_opinion,no_stop_words_majority
0,5d66e470e59ddf33befdfe06,None,37588,In the matter of the Complaint made against Jo...,"[{'text': 'ORDER OF PUBLIC CENSURE Whereas, In...",8874,Kansas Supreme Court,1970,https://cite.case.law/kan/206/335/,None,order of public censure whereas in a proceedin...,[],"(order, of, public, censure, whereas, in, a, p...","[order, public, censure, proceeding, conducted..."
1,5d66e470e59ddf33befdfe07,None,37656,In the matter of the Complaint made against Ro...,"[{'text': 'ORDER OF PUBLIC CENSURE Whereas, In...",8874,Kansas Supreme Court,1970,https://cite.case.law/kan/206/334/,None,order of public censure whereas in a proceedin...,[],"(order, of, public, censure, whereas, in, a, p...","[order, public, censure, proceeding, conducted..."
2,5d66e470e59ddf33befdfe08,None,1747634,State of Connecticut v. Anonymous (1971-5),"[{'text': 'Moraghan, J. The defendant has move...",13405,Connecticut Circuit Court,1970,https://cite.case.law/conn-cir-ct/6/443/,None,moraghan j the defendant has moved to suppress...,[],"(moraghan, j, the, defendant, has, moved, to, ...","[moraghan, j, defendant, moved, suppress, cert..."
3,5d66e470e59ddf33befdfe09,None,1747661,State of Connecticut v. Anonymous (1971-3),"[{'text': 'Dearington, J. Following a trial to...",13405,Connecticut Circuit Court,1970,https://cite.case.law/conn-cir-ct/6/393/,None,dearington j following a trial to the jury the...,[],"(dearington, j, following, a, trial, to, the, ...","[dearington, j, following, trial, jury, defend..."
4,5d66e470e59ddf33befdfe0a,None,1747680,State of Connecticut v. Anonymous (1971-7),"[{'text': 'Casale, J. The defendant, after a t...",13405,Connecticut Circuit Court,1970,https://cite.case.law/conn-cir-ct/6/462/,None,casale j the defendant after a trial to the co...,[],"(casale, j, the, defendant, after, a, trial, t...","[casale, j, defendant, trial, court, found, gu..."


In [917]:
clean_data_df._id = clean_data_df._id.apply(lambda x : str(x))

In [10]:
#text in tokenized are spacy tokens, so have to convert them to strings for the models 
majority = [i.text for i in test.tokenized_majority_opinion]

In [918]:
clean_data_df.to_csv('clean_data_df_15000.csv')

***Doc2Vec Model***

In [919]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [920]:
list_of_token_text = list(sent_to_words(clean_data_df.tokenized_majority_opinion))

In [921]:
#tag each document 
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(list_of_token_text)]
model = Doc2Vec(documents, vector_size=300, window=1, 
                min_count=1, workers=4, dm=1, 
                sample= 0.01, dm_concat = 1, negative = 5, dbow_words = 1)


In [922]:
model.save("model_15000.model")

In [33]:
model.most_similar('woman')

[('boy', 0.7236981391906738),
 ('man', 0.6970038414001465),
 ('girl', 0.6482921838760376),
 ('daughter', 0.6438713073730469),
 ('seaman', 0.6122671961784363),
 ('horse', 0.6120308637619019),
 ('female', 0.6110771894454956),
 ('legatee', 0.6098716259002686),
 ('women', 0.6035140752792358),
 ('codefendant', 0.6034492254257202)]

In [924]:
model.most_similar('black')

[('white', 0.6352797150611877),
 ('nude', 0.5251858830451965),
 ('negro', 0.5092548131942749),
 ('colored', 0.49261340498924255),
 ('male', 0.49129781126976013),
 ('cisek', 0.4821275472640991),
 ('christian', 0.47875675559043884),
 ('bird', 0.4691063463687897),
 ('pringle', 0.4666266143321991),
 ('younger', 0.45124179124832153)]

In [34]:
def analogy(x1, x2, y1):
    result = model.most_similar(positive=[y1, x2], negative=[x1])
    return result[0][0]

In [35]:
analogy('woman', 'rape', 'man')

'murder'

***Recommendation Engine Using Doc2Vec as Features***

In [328]:
test_case_date = input('insert case year as four digits, i.e. 2018: ')
test_case_date = int(test_case_date)

insert case year as four digits, i.e. 2018: 2018


In [827]:
test_input = input('insert case text: ')

insert case text: Case Text: District of Columbia law bans handgun possession by making it a crime to carry an unregistered firearm and prohibiting the registration of handguns; provides separately that no person may carry an unlicensed handgun, but authorizes the police chief to issue 1-year licenses; and requires residents to keep lawfully owned firearms unloaded and disassembled or bound by a trigger lock or similar device. Respondent Heller, a D. C. special policeman, applied to register a handgun he wished to keep at home, but the District refused. He filed this suit seeking, on Second Amendment grounds, to enjoin the city from enforcing the bar on handgun registration, the licensing requirement insofar as it prohibits carrying an unlicensed firearm in the home, and the trigger-lock requirement insofar as it prohibits the use of functional firearms in the home. The District Court dismissed the suit, but the D. C. Circuit reversed, holding that the Second Amendment protects an indi

In [828]:
def clean_test_input(test_text):
    test_input = test_text.lower().translate(str.maketrans('  ', '  ', string.punctuation)).replace('\n', ' ').replace('•', ' ').replace('“', ' ').replace('”', ' ')
    test_input = nlp.tokenizer(test_input) 
    test_input = c_d.stop_word_remove(test_input)
    test_vector = model.infer_vector(test_input)
    top_10_similar_cases = model.docvecs.most_similar([test_vector])
    input_vector = pd.DataFrame(test_vector).T
    return input_vector

In [830]:
test_input = test_input.lower().translate(str.maketrans('  ', '  ', string.punctuation)).replace('\n', ' ').replace('•', ' ').replace('“', ' ').replace('”', ' ')
test_input = nlp.tokenizer(test_input) 
test_input = c_d.stop_word_remove(test_input)
test_vector = model.infer_vector(test_input)
top_10_similar_cases = model.docvecs.most_similar([test_vector])

In [831]:
top_10_similar_cases 

[(6220, 0.7356553673744202),
 (5680, 0.7268011569976807),
 (7205, 0.7000726461410522),
 (7293, 0.6920267343521118),
 (6021, 0.6893383264541626),
 (4591, 0.6887164115905762),
 (3042, 0.6844009160995483),
 (3575, 0.6843631267547607),
 (2015, 0.6799923777580261),
 (5221, 0.6762263774871826)]

In [833]:
clean_data_df.columns

Index(['_id', 'attorneys', 'case_id', 'case_name', 'case_text', 'court_id',
       'court_name', 'decision_date', 'frontend_url', 'judges',
       'majority_opinion', 'other_opinions', 'tokenized_majority_opinion',
       'no_stop_words_majority'],
      dtype='object')

In [834]:
clean_data_df['majority_opinion'][5680]

'craven circuit judge lynn porter was convicted of occupying a house containing gambling paraphernalia in violation of sc code of laws § 165152 1962 the south carolina supreme court affirmed his conviction state v porter 251 sc 393 162 se2d 843 1968 cert denied 393 us 1079 89 sct 859 21 led2d 773 1969 on petition for a writ of habeas corpus the district court adjudged porter’s conviction unconstitu tional on the ground that a search of his home incident to the arrest was overbroad the court held that the unconstitutional search in turn required the exclusion of evidence seized and offered to prove porter’s guilt the state appealed we reverse ted m owens an agent with the south carolina law enforcement division was assigned to investigate gambling activity in greenville county south carolina on august 24 1965 he testified at porter’s trial that he received from a reliable source the telephone number of a party accepting bets on baseball and football games he called the number and began 

In [819]:
input_vector = clean_test_input(test_input)

In [820]:
input_vector

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.13188,0.027174,-0.104284,-0.081959,0.041408,0.045122,0.09816,-0.050489,-0.118886,0.122865,...,0.152269,0.250656,-0.045469,0.089309,-0.048416,0.014992,0.089043,-0.032066,0.074593,-0.171983


In [925]:
def docvec_df(model_docvecs):
    _list = []

    for i in range(len(model_docvecs)):
        test = pd.DataFrame({clean_data_df['_id'][i]: list(model_docvecs[i])}).T
        _list.append(test)
    
    vec_features = pd.concat(_list, sort = False)
    return vec_features

In [926]:
vec_features = docvec_df(model.docvecs)

In [927]:
len(vec_features)

14663

In [928]:
def date_df(clean_df):
    feature_df = clean_df
    feature_df = feature_df.drop(columns = ['attorneys', 'case_name', 'case_text', 'frontend_url', 'judges',
           'majority_opinion', 'other_opinions', 'tokenized_majority_opinion',
                                            'no_stop_words_majority'])
    feature_df['decision_date'] = feature_df['decision_date'].apply(lambda x : int(x[:4]))
    return feature_df

In [929]:
feature_df = date_df(clean_data_df)
feature_df.head()

,_id,case_id,court_id,court_name,decision_date
0,5d66e470e59ddf33befdfe06,37588,8874,Kansas Supreme Court,1970
1,5d66e470e59ddf33befdfe07,37656,8874,Kansas Supreme Court,1970
2,5d66e470e59ddf33befdfe08,1747634,13405,Connecticut Circuit Court,1970
3,5d66e470e59ddf33befdfe09,1747661,13405,Connecticut Circuit Court,1970
4,5d66e470e59ddf33befdfe0a,1747680,13405,Connecticut Circuit Court,1970


In [930]:
def combine_df(df1, df2):
    combined_df = feature_df.merge(vec_features, left_on='_id', right_index= True, copy=False)
    combined_df.drop(['case_id', '_id', 'court_id', 'court_name'], axis = 1, inplace = True)
    return combined_df

In [934]:
combined_df.to_csv('combined_df_15000.csv')

In [932]:
combined_df = combine_df(feature_df, vec_features)
combined_df.head()

,decision_date,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,1970,0.055578,0.062793,0.111757,-0.050747,0.022060,-0.029708,0.086982,0.033576,-0.071184,...,0.129361,0.162560,0.046071,-0.040822,-0.143222,-0.057379,0.014555,-0.123648,0.150177,-0.010832
1,1970,0.036769,0.018969,0.117687,0.012926,0.055218,-0.064043,0.064138,0.050911,-0.080674,...,0.149662,0.132872,-0.013231,-0.051893,-0.137733,-0.087553,0.014746,-0.084734,0.129504,0.019240
2,1970,0.173926,0.046345,-0.039223,-0.046244,-0.178465,0.102620,0.233139,0.056684,-0.008768,...,0.240780,-0.026236,0.014886,0.031901,-0.064725,-0.306861,0.132542,-0.202208,0.094480,0.024635
3,1970,0.011757,-0.064485,-0.256493,0.101125,0.051026,0.011756,0.084242,0.068421,0.132079,...,-0.101361,0.153977,-0.066488,-0.133268,-0.126043,0.068069,-0.013512,-0.100623,0.178657,-0.074497
4,1970,-0.011718,-0.079580,0.011535,0.134300,-0.113267,0.014437,0.079903,0.206433,0.097947,...,0.105243,0.199643,-0.055408,-0.166839,-0.040514,0.023714,0.004094,-0.029682,0.049198,0.080851


In [933]:
combined_df.shape

(14663, 301)

In [907]:
def cosine_text_similarity(text_vector, case_date, combined_df, clean_data_df):
    alpha = -0.07 #weight of date and type of court

    text_similarity = cosine_similarity(combined_df.iloc[:, 1:], text_vector)
    rec_df = pd.DataFrame({'_id': clean_data_df['_id'], 'date': combined_df['decision_date'], 'similarity': list(text_similarity)})
    rec_df['similarity'] = rec_df['similarity'].apply(lambda x: x[0])
    rec_df['date_similarity'] = rec_df['date'].apply(lambda x: abs(x - case_date)/ 100)
    rec_df['tot_similarity'] = rec_df['similarity'] + (alpha * rec_df['date_similarity'])
    rec_df = rec_df.sort_values(by = 'tot_similarity', ascending=False).reset_index().loc[0:15]
    rec_df.drop(['index'], axis = 1, inplace = True)

    _ids = []
    for i in rec_df['_id']:
        _ids.append(i)

    #print(_ids)
    _index_values = []
    for i in _ids:
        temp = (str(clean_data_df.loc[clean_data_df['_id'] == f'{i}']['case_name']))
        temp2 = temp.split()[0]
        _index_values.append(int(temp2))

    case_list = []
    for j, i in enumerate(_index_values):
        case_list.append(f"{j+1} -- {clean_data_df['case_name'][i]}")       

        case_list.append(f"{clean_data_df['majority_opinion'][i]}")
        
    
    return(case_list)
        
        
        
        
        

In [908]:
text = cosine_text_similarity(input_vector, 1980, combined_df, clean_data_df)

In [909]:
text[0]

'1 -- Neale WOOD, a minor, by his mother and next friend, Katrina Wood, Plaintiffs, v. ALAMO HEIGHTS INDEPENDENT SCHOOL DISTRICT et al., Defendants'

In [912]:
text[3]

'ingraham circuit judge sitting by designation m emorandum  this action was filed pursuant to 42 usc sec 1983 and 28 usc sec 2201 by three students at bellaire high school against the school principal and the superintendent and members of the board of the houston independent school district the plaintiffs allege that as a result of their publishing and distributing an offcampus publication called the plain brown watermelon they were harassed by school officials and were told to leave the school until their attitudes changed contending that their rights under the first fifth and fourteenth amendments to the united states constitution were violated the plaintiffs first sought a temporary restraining order to restrain the defendants or their agents from refusing the plaintiffs permission to reenter the school from harassing the plaintiffs and from imposing discipline upon them because of their activities in connection with the newspaper the motion for the temporary restraining order was d